In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Описание

[Датасет](https://www.kaggle.com/rajeevw/ufcdata) представляет собой полный список всех боёв **UFC**. Каждая строка содержит информацию об обоих бойцах, деталях боя и о том, кто победил. 

Весь датасет состоит из 4-х файлов:
* `data.csv` - список всех боёв UFC и информация о бое
* `raw_fighter_details.csv` - список всех бойцов UFC за эти годы и информация о них
* `preprocessed_data.csv` - обработанные данные, содержащие всю необходимую информацию для составления прогнозов
* `raw_total_fight_data.csv` - необработанные данные для того, чтобы можно было составить и обработать данные самим

При анализе и составлении модели будем ориентироваться на датасет `preprocessed_data.csv`.

Каждая строка содержит статистику обоих бойцов. Бойцы представлены `red` и `blue` углами. К примеру, в данных указана средняя статистика красного бойца по всем боям, кроме текущего. Столбец `Winner` указывает на победителя - целевая переменная. 

**Описание признаков**:

* `R_` и `B_` - префикс, указывающий на то, к какому бойцу относится статистика 
* `_opp_` - указывает но то, какой урон нанесен оппонентом бойца
* `KD` - число нокдаунов
* `SIG_STR` - число существенных ударов
* `SIG_STR_pct` - процент существенных ударов
* `TOTAL_STR` - всего ударов (достигших цели от общего числа нанесённых - landed of attempted)
* `TD` - число тейкдаунов
* `TD_pct` - процент тейкдаунов
* `SUB_ATT` - число попыток болевого приёма
* `PASS` - "times the guard was passed?"
* `REV` - is the no. of Reversals landed
* `HEAD` - число существенных ударов в голову
* `BODY` - число существенных ударов в тело
* `CLINCH` - число существенных ударов в клинче
* `GROUND` - число существеных ударов "на земле"
* `win_by` - чем получена победа
* `last_round` - последний раунд в бое (если нокаут в первом раунде, то это 1)
* `last_round_time` - время, когда бой закончился в последнем раунде
* `Format` - формат боя (3 раунда, 5 раундов и т.д.)
* `Referee` - имя судьи
* `date` - дата боя
* `location` - локация боя
* `Fight_type` - весовая категория и титульный ли это бой
* `Winner` - победитель боя
* `Stance` - позиция бойца
* `Height_cms` - высота в сантиметрах
* `Reach_cms` - длина рук в сантиметрах
* `Weight_lbs`  - вес в фунтах
* `age` - возраст бойца
* `title_bout` - титульный ли бой
* `weight_class` - весовая категория 
* `no_of_rounds` - количество раундов, на которое запланирован бой
* `current_lose_streak` - текущая проигрышная серия
* `current_win_streak` - текущая выигрышная серия
* `draw` - количество ничей в карьере
* `wins` - количество побед в карьере
* `losses` - количество проигрышей в карьере
* `total_rounds_fought` - среднее количество раундов
* `total_time_fought(seconds)` - общее количество времени в бое
* `total_title_bouts` - общее количество титульных боёв, в которых принято участие
* `win_by_Decision_Majority` - количество побед в карьере, полученных по решению большинства судей
* `win_by_Decision_Split` - количество побед в карьере, полученных по решению разеделенным мненением судей
* `win_by_Decision_Unanimous` - количество побед в карьере, полученных единогласным решением судей
* `win_by_KO/TKO` - количество побед нокаутом
* `win_by_Submission` - количество побед болевым приёмом
* `win_by_TKO_Doctor_Stoppage` - количество побед, полученных за счёт остановки боя доктором


Возникает объективная гипотеза о том, какие признаки действительно являются существенными в определении победителя: 
- возраст бойца
- длина рук
- рост
- вес
- каким стилем дерётся - ударник или борец
- опыт
- насколько боец техничнее своего оппонента

Чтобы выявить, какие признаки являются существенными, попробуем обучить случайный лес и убедиться. Важное правило: **мусор на входе - мусор на выходе**.

In [2]:
df = pd.read_csv('./preprocessed_data.csv')
df

,Winner,title_bout,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,Red,False,0.000000,0.0,0.420000,0.49500,0.330,0.36000,0.500000,1.000000,...,0,1,0,0,0,0,1,0,0,0
1,Red,False,0.500000,0.0,0.660000,0.30500,0.300,0.50000,1.500000,0.000000,...,0,1,0,0,0,0,1,0,0,0
2,Red,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,1,0,0,0,0,0,0,1,0
3,Blue,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,0,0,1,0,0,0,0,0,1
4,Blue,False,0.125000,0.0,0.535625,0.57875,0.185,0.16625,0.125000,0.187500,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,Red,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,1,0,0,0,0,0,0,1,0
5898,Red,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,1,0,0,0,0,0,0,1,0
5899,Red,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,1,0,0,0,0,1,0,0,0
5900,Red,False,0.015625,0.0,0.450000,0.42750,0.250,0.20000,0.148468,0.098389,...,0,1,0,0,0,0,1,0,0,0


Полный список признаков:

In [3]:
list(df.columns)

['Winner',
 'title_bout',
 'B_avg_KD',
 'B_avg_opp_KD',
 'B_avg_SIG_STR_pct',
 'B_avg_opp_SIG_STR_pct',
 'B_avg_TD_pct',
 'B_avg_opp_TD_pct',
 'B_avg_SUB_ATT',
 'B_avg_opp_SUB_ATT',
 'B_avg_REV',
 'B_avg_opp_REV',
 'B_avg_SIG_STR_att',
 'B_avg_SIG_STR_landed',
 'B_avg_opp_SIG_STR_att',
 'B_avg_opp_SIG_STR_landed',
 'B_avg_TOTAL_STR_att',
 'B_avg_TOTAL_STR_landed',
 'B_avg_opp_TOTAL_STR_att',
 'B_avg_opp_TOTAL_STR_landed',
 'B_avg_TD_att',
 'B_avg_TD_landed',
 'B_avg_opp_TD_att',
 'B_avg_opp_TD_landed',
 'B_avg_HEAD_att',
 'B_avg_HEAD_landed',
 'B_avg_opp_HEAD_att',
 'B_avg_opp_HEAD_landed',
 'B_avg_BODY_att',
 'B_avg_BODY_landed',
 'B_avg_opp_BODY_att',
 'B_avg_opp_BODY_landed',
 'B_avg_LEG_att',
 'B_avg_LEG_landed',
 'B_avg_opp_LEG_att',
 'B_avg_opp_LEG_landed',
 'B_avg_DISTANCE_att',
 'B_avg_DISTANCE_landed',
 'B_avg_opp_DISTANCE_att',
 'B_avg_opp_DISTANCE_landed',
 'B_avg_CLINCH_att',
 'B_avg_CLINCH_landed',
 'B_avg_opp_CLINCH_att',
 'B_avg_opp_CLINCH_landed',
 'B_avg_GROUND_att',
 

In [18]:
X = df.drop('Winner', axis=1)
y = df['Winner']

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators": [100, 500],
     "criterion": ["gini"],
     "max_depth": [10],
     "min_samples_split": [2],
     "min_samples_leaf": [2]}
]

model = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, n_jobs=-1, verbose=3, scoring='f1_weighted')

In [44]:
model.fit(X, y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'criterion': ['gini'], 'max_depth': [10],
                          'min_samples_leaf': [2], 'min_samples_split': [2],
                          'n_estimators': [100, 500]}],
             scoring='f1_weighted', verbose=3)

In [45]:
model.best_score_

0.5916527212122004

In [48]:
forest = model.best_estimator_

for feature, importance in sorted(zip(df.drop('Winner', axis=1).columns, forest.feature_importances_), key=lambda x: x[1], reverse=True):
    print(feature, importance)

R_age 0.02020299487723125
R_avg_opp_HEAD_landed 0.017030059391012906
R_avg_opp_DISTANCE_landed 0.01663801256203886
B_avg_SIG_STR_att 0.0156601955029509
R_avg_opp_SIG_STR_landed 0.015554648671869849
B_avg_HEAD_att 0.015479057792121248
B_avg_DISTANCE_landed 0.014300407210805773
R_avg_opp_SIG_STR_pct 0.013788313837016659
B_avg_SIG_STR_landed 0.012964961882834037
B_avg_DISTANCE_att 0.012820604054729807
R_avg_opp_BODY_att 0.012265527315616524
R_avg_GROUND_att 0.012223697807103466
B_avg_HEAD_landed 0.012221334292198873
R_avg_opp_CTRL_time(seconds) 0.011596363555431501
B_avg_TD_att 0.01098767319451979
B_avg_BODY_att 0.010943958380532725
B_avg_opp_DISTANCE_landed 0.010866650361211469
R_losses 0.010796015445459078
R_avg_CTRL_time(seconds) 0.01049127567522972
R_avg_GROUND_landed 0.010487857217910047
B_age 0.010354521629402848
R_avg_SIG_STR_pct 0.010163764747388072
R_avg_opp_BODY_landed 0.009842237211126259
R_avg_opp_SIG_STR_att 0.009770613124076198
R_avg_opp_TOTAL_STR_landed 0.009662553615575548

Модель случайного леса является лишь немного лучше случайного классификатора, поэтому гарантировать, что она даёт правильную важность признаков так же не стоит. Однако, из списка важных признаков можно заметить вполне логичные вещи:
- весовая категория бойцов
- количество ничей
- количество побед 